In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """read data from data file"""
    with open(filename, "rb") as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1
        self._labels = np.hstack(all_labels)
        #print(self._data.shape)
        #print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
                    
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' %i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

#batch_data, batch_labels = train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)

In [6]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

x = tf.compat.v1.placeholder(tf.float32, [None,3072])
y = tf.compat.v1.placeholder(tf.int64, [None])

w = tf.compat.v1.get_variable('w', [x.get_shape()[-1], 10], initializer=tf.random_normal_initializer(0,1))
b = tf.compat.v1.get_variable('b', [10], initializer= tf.constant_initializer(0.0))
y_ = tf.matmul(x, w) + b
p_y = tf.nn.softmax(y_)
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))

predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.compat.v1.train.AdamOptimizer(1e-3).minimize(loss)

In [8]:
init = tf.compat.v1.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x:batch_data, y:batch_labels})
        if (i+1) %500 == 0:
            print ("[Train] Step: %d, loss: %4.5f, acc:%4.5f" %(i, loss_val, acc_val))
        if (i+1) %5000 ==0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
                test_acc=np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' %(i+1, test_acc))
            

[Train] Step: 499, loss: 0.13937, acc:0.25000
[Train] Step: 999, loss: 0.15853, acc:0.20000
[Train] Step: 1499, loss: 0.13772, acc:0.30000
[Train] Step: 1999, loss: 0.15160, acc:0.20000
[Train] Step: 2499, loss: 0.14122, acc:0.25000
[Train] Step: 2999, loss: 0.16797, acc:0.15000
[Train] Step: 3499, loss: 0.14842, acc:0.25000
[Train] Step: 3999, loss: 0.15732, acc:0.20000
[Train] Step: 4499, loss: 0.14688, acc:0.25000
[Train] Step: 4999, loss: 0.13043, acc:0.35000
[Test] Step: 5000, acc: 0.55000
[Test] Step: 5000, acc: 0.42500
[Test] Step: 5000, acc: 0.36667
[Test] Step: 5000, acc: 0.36250
[Test] Step: 5000, acc: 0.36000
[Test] Step: 5000, acc: 0.36667
[Test] Step: 5000, acc: 0.35714
[Test] Step: 5000, acc: 0.35000
[Test] Step: 5000, acc: 0.36667
[Test] Step: 5000, acc: 0.34500
[Test] Step: 5000, acc: 0.34545
[Test] Step: 5000, acc: 0.34167
[Test] Step: 5000, acc: 0.31923
[Test] Step: 5000, acc: 0.31071
[Test] Step: 5000, acc: 0.32000
[Test] Step: 5000, acc: 0.32500
[Test] Step: 5000, a

[Train] Step: 15499, loss: 0.12527, acc:0.35000
[Train] Step: 15999, loss: 0.13882, acc:0.30000
[Train] Step: 16499, loss: 0.12728, acc:0.35000
[Train] Step: 16999, loss: 0.08938, acc:0.55000
[Train] Step: 17499, loss: 0.09791, acc:0.50000
[Train] Step: 17999, loss: 0.11851, acc:0.40000
[Train] Step: 18499, loss: 0.13985, acc:0.30000
[Train] Step: 18999, loss: 0.12851, acc:0.35000
[Train] Step: 19499, loss: 0.12998, acc:0.35000
[Train] Step: 19999, loss: 0.13725, acc:0.30000
[Test] Step: 20000, acc: 0.50000
[Test] Step: 20000, acc: 0.42500
[Test] Step: 20000, acc: 0.38333
[Test] Step: 20000, acc: 0.37500
[Test] Step: 20000, acc: 0.37000
[Test] Step: 20000, acc: 0.36667
[Test] Step: 20000, acc: 0.35000
[Test] Step: 20000, acc: 0.35625
[Test] Step: 20000, acc: 0.36667
[Test] Step: 20000, acc: 0.36000
[Test] Step: 20000, acc: 0.35455
[Test] Step: 20000, acc: 0.34583
[Test] Step: 20000, acc: 0.33846
[Test] Step: 20000, acc: 0.32857
[Test] Step: 20000, acc: 0.34000
[Test] Step: 20000, acc: 

[Train] Step: 30499, loss: 0.04987, acc:0.75000
[Train] Step: 30999, loss: 0.09573, acc:0.45000
[Train] Step: 31499, loss: 0.14742, acc:0.25000
[Train] Step: 31999, loss: 0.09938, acc:0.50000
[Train] Step: 32499, loss: 0.14612, acc:0.20000
[Train] Step: 32999, loss: 0.06992, acc:0.65000
[Train] Step: 33499, loss: 0.12133, acc:0.40000
[Train] Step: 33999, loss: 0.14770, acc:0.25000
[Train] Step: 34499, loss: 0.11597, acc:0.40000
[Train] Step: 34999, loss: 0.13456, acc:0.30000
[Test] Step: 35000, acc: 0.60000
[Test] Step: 35000, acc: 0.47500
[Test] Step: 35000, acc: 0.43333
[Test] Step: 35000, acc: 0.41250
[Test] Step: 35000, acc: 0.41000
[Test] Step: 35000, acc: 0.40833
[Test] Step: 35000, acc: 0.40000
[Test] Step: 35000, acc: 0.40000
[Test] Step: 35000, acc: 0.40000
[Test] Step: 35000, acc: 0.40000
[Test] Step: 35000, acc: 0.38636
[Test] Step: 35000, acc: 0.38333
[Test] Step: 35000, acc: 0.37308
[Test] Step: 35000, acc: 0.36786
[Test] Step: 35000, acc: 0.38000
[Test] Step: 35000, acc: 

[Train] Step: 45499, loss: 0.13819, acc:0.30000
[Train] Step: 45999, loss: 0.09993, acc:0.50000
[Train] Step: 46499, loss: 0.10535, acc:0.45000
[Train] Step: 46999, loss: 0.11676, acc:0.35000
[Train] Step: 47499, loss: 0.12358, acc:0.35000
[Train] Step: 47999, loss: 0.11857, acc:0.40000
[Train] Step: 48499, loss: 0.11972, acc:0.40000
[Train] Step: 48999, loss: 0.12849, acc:0.35000
[Train] Step: 49499, loss: 0.10036, acc:0.50000
[Train] Step: 49999, loss: 0.07992, acc:0.60000
[Test] Step: 50000, acc: 0.55000
[Test] Step: 50000, acc: 0.45000
[Test] Step: 50000, acc: 0.43333
[Test] Step: 50000, acc: 0.45000
[Test] Step: 50000, acc: 0.42000
[Test] Step: 50000, acc: 0.40000
[Test] Step: 50000, acc: 0.39286
[Test] Step: 50000, acc: 0.39375
[Test] Step: 50000, acc: 0.40000
[Test] Step: 50000, acc: 0.38500
[Test] Step: 50000, acc: 0.37727
[Test] Step: 50000, acc: 0.37083
[Test] Step: 50000, acc: 0.36154
[Test] Step: 50000, acc: 0.35714
[Test] Step: 50000, acc: 0.37333
[Test] Step: 50000, acc: 

[Train] Step: 60499, loss: 0.08600, acc:0.55000
[Train] Step: 60999, loss: 0.08230, acc:0.55000
[Train] Step: 61499, loss: 0.15375, acc:0.20000
[Train] Step: 61999, loss: 0.06625, acc:0.65000
[Train] Step: 62499, loss: 0.10489, acc:0.45000
[Train] Step: 62999, loss: 0.11825, acc:0.40000
[Train] Step: 63499, loss: 0.12125, acc:0.40000
[Train] Step: 63999, loss: 0.08857, acc:0.55000
[Train] Step: 64499, loss: 0.10670, acc:0.45000
[Train] Step: 64999, loss: 0.14855, acc:0.25000
[Test] Step: 65000, acc: 0.60000
[Test] Step: 65000, acc: 0.47500
[Test] Step: 65000, acc: 0.41667
[Test] Step: 65000, acc: 0.41250
[Test] Step: 65000, acc: 0.40000
[Test] Step: 65000, acc: 0.38333
[Test] Step: 65000, acc: 0.38571
[Test] Step: 65000, acc: 0.38125
[Test] Step: 65000, acc: 0.37778
[Test] Step: 65000, acc: 0.36000
[Test] Step: 65000, acc: 0.34545
[Test] Step: 65000, acc: 0.35000
[Test] Step: 65000, acc: 0.34231
[Test] Step: 65000, acc: 0.33571
[Test] Step: 65000, acc: 0.35000
[Test] Step: 65000, acc: 

[Train] Step: 75499, loss: 0.11701, acc:0.40000
[Train] Step: 75999, loss: 0.08827, acc:0.55000
[Train] Step: 76499, loss: 0.10179, acc:0.45000
[Train] Step: 76999, loss: 0.10920, acc:0.45000
[Train] Step: 77499, loss: 0.11270, acc:0.40000
[Train] Step: 77999, loss: 0.10498, acc:0.45000
[Train] Step: 78499, loss: 0.12428, acc:0.35000
[Train] Step: 78999, loss: 0.10808, acc:0.45000
[Train] Step: 79499, loss: 0.09945, acc:0.50000
[Train] Step: 79999, loss: 0.06620, acc:0.65000
[Test] Step: 80000, acc: 0.55000
[Test] Step: 80000, acc: 0.45000
[Test] Step: 80000, acc: 0.40000
[Test] Step: 80000, acc: 0.40000
[Test] Step: 80000, acc: 0.40000
[Test] Step: 80000, acc: 0.38333
[Test] Step: 80000, acc: 0.37143
[Test] Step: 80000, acc: 0.37500
[Test] Step: 80000, acc: 0.37778
[Test] Step: 80000, acc: 0.36500
[Test] Step: 80000, acc: 0.35909
[Test] Step: 80000, acc: 0.36250
[Test] Step: 80000, acc: 0.35000
[Test] Step: 80000, acc: 0.34286
[Test] Step: 80000, acc: 0.35667
[Test] Step: 80000, acc: 

[Train] Step: 90499, loss: 0.05430, acc:0.70000
[Train] Step: 90999, loss: 0.07020, acc:0.60000
[Train] Step: 91499, loss: 0.13635, acc:0.30000
[Train] Step: 91999, loss: 0.07022, acc:0.65000
[Train] Step: 92499, loss: 0.11767, acc:0.40000
[Train] Step: 92999, loss: 0.10514, acc:0.45000
[Train] Step: 93499, loss: 0.08266, acc:0.55000
[Train] Step: 93999, loss: 0.09831, acc:0.50000
[Train] Step: 94499, loss: 0.10112, acc:0.50000
[Train] Step: 94999, loss: 0.10069, acc:0.45000
[Test] Step: 95000, acc: 0.55000
[Test] Step: 95000, acc: 0.42500
[Test] Step: 95000, acc: 0.38333
[Test] Step: 95000, acc: 0.40000
[Test] Step: 95000, acc: 0.40000
[Test] Step: 95000, acc: 0.37500
[Test] Step: 95000, acc: 0.37857
[Test] Step: 95000, acc: 0.37500
[Test] Step: 95000, acc: 0.36667
[Test] Step: 95000, acc: 0.35500
[Test] Step: 95000, acc: 0.35455
[Test] Step: 95000, acc: 0.35417
[Test] Step: 95000, acc: 0.33846
[Test] Step: 95000, acc: 0.32857
[Test] Step: 95000, acc: 0.34000
[Test] Step: 95000, acc: 